In [1]:
%%configure
{"conf": {
    "spark.app.name": "datavirus_connections"
}}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7932,application_1589299642358_2450,pyspark,idle,Link,Link,
7933,application_1589299642358_2451,pyspark,idle,Link,Link,
7935,application_1589299642358_2453,pyspark,idle,Link,Link,
7936,application_1589299642358_2454,pyspark,idle,Link,Link,
7937,application_1589299642358_2455,pyspark,busy,Link,Link,
7938,application_1589299642358_2456,pyspark,idle,Link,Link,
7939,application_1589299642358_2457,pyspark,idle,Link,Link,
7940,application_1589299642358_2458,pyspark,idle,Link,Link,
7941,application_1589299642358_2459,pyspark,idle,Link,Link,
7942,application_1589299642358_2460,pyspark,idle,Link,Link,


In [2]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7946,application_1589299642358_2464,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7932,application_1589299642358_2450,pyspark,idle,Link,Link,
7933,application_1589299642358_2451,pyspark,idle,Link,Link,
7935,application_1589299642358_2453,pyspark,idle,Link,Link,
7936,application_1589299642358_2454,pyspark,idle,Link,Link,
7937,application_1589299642358_2455,pyspark,busy,Link,Link,
7938,application_1589299642358_2456,pyspark,idle,Link,Link,
7939,application_1589299642358_2457,pyspark,idle,Link,Link,
7940,application_1589299642358_2458,pyspark,idle,Link,Link,
7941,application_1589299642358_2459,pyspark,idle,Link,Link,
7942,application_1589299642358_2460,pyspark,idle,Link,Link,


In [4]:
import pyspark.sql.functions as f

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
probability = spark.read.orc('/user/gflueck/probability.orc')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
probability.show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+---------------+----------+--------------+-------+---------+------------------+--------------------+
|arrival_time_minute|        trip_id|station_id|transport_type|line_id|line_text| delay_probability|     delay_parameter|
+-------------------+---------------+----------+--------------+-------+---------+------------------+--------------------+
|              21:42|   85:11:10:002|   8503202|           Zug|     10|       EC|0.8202508288487688|0.014650408388357685|
|              02:58|85:11:13714:001|   8503000|           Zug|  13714|      SN1|0.6519774011299435|0.006361772034664491|
+-------------------+---------------+----------+--------------+-------+---------+------------------+--------------------+
only showing top 2 rows

In [7]:
schedule = spark.read.orc('/user/datavirus/schedule_zurich.orc')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
schedule.show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+--------------------+----------+-------------+---------+------------+
|Stop_sequence|             Trip_ID|Start_Time|Start_Station|Stop_Time|Stop_Station|
+-------------+--------------------+----------+-------------+---------+------------+
|           19|299.TA.26-308-j19...|  15:51:00|      8591833| 15:52:00|     8590833|
|           20|299.TA.26-308-j19...|  15:52:00|      8590833| 15:53:00|     8590727|
+-------------+--------------------+----------+-------------+---------+------------+
only showing top 2 rows

In [57]:
trips = spark.read.orc("/data/sbb/timetables/orc/trips/")
calendar = spark.read.orc("/data/sbb/timetables/orc/calendar/")
routes = spark.read.orc("/data/sbb/timetables/orc/routes/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [73]:
trips.show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----------+--------------------+------------------+---------------+------------+
|   route_id|service_id|             trip_id|     trip_headsign|trip_short_name|direction_id|
+-----------+----------+--------------------+------------------+---------------+------------+
|1-1-C-j19-1|  TA+b0001|5.TA.1-1-C-j19-1.3.R|Zofingen, Altachen|            108|           1|
|1-1-C-j19-1|  TA+b0001|7.TA.1-1-C-j19-1.3.R|Zofingen, Altachen|            112|           1|
+-----------+----------+--------------------+------------------+---------------+------------+
only showing top 2 rows

In [23]:
monday_service = (
    calendar
    .where(calendar.monday == True)
    .select('service_id')
    .dropDuplicates()
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
monday_trips = (
    trips
    .join(f.broadcast(monday_service), 'service_id')
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:
route_trips = (
    trips
    .join(routes, 'route_id')
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [72]:
route_trips.where('agency_id == "11"').orderBy(['agency_id', 'trip_short_name']).dropDuplicates(['agency_id', 'trip_short_name']).show(20, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+----------+------------------------+---------------+---------------+------------+---------+----------------+---------------+----------+----------+
|route_id      |service_id|trip_id                 |trip_headsign  |trip_short_name|direction_id|agency_id|route_short_name|route_long_name|route_desc|route_type|
+--------------+----------+------------------------+---------------+---------------+------------+---------+----------------+---------------+----------+----------+
|40-18-Y-j19-1 |TA+b06uw  |25.TA.40-18-Y-j19-1.22.H|Zürich HB      |10             |0           |11       |18-Y            |               |Eurocity  |102       |
|54-2-Y-j19-1  |TA+b046m  |13.TA.54-2-Y-j19-1.11.H |Basel Bad Bf   |100            |0           |11       |2-Y             |               |ICE       |102       |
|5-3-j19-1     |TA+b01i5  |161.TA.5-3-j19-1.79.H   |Zürich HB      |10003          |0           |11       |3               |               |Intercity |102       |
|54-4-Y-j19-1  |TA+b02

In [70]:
probability.withColumn('operator', operator(probability.trip_id)).where('operator == "11"').dropDuplicates(['line_id']).orderBy(['line_id', 'line_text']).show(100, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+---------------+----------+--------------+-------+---------+------------------+---------------------+--------+
|arrival_time_minute|trip_id        |station_id|transport_type|line_id|line_text|delay_probability |delay_parameter      |operator|
+-------------------+---------------+----------+--------------+-------+---------+------------------+---------------------+--------+
| 21:42             |85:11:10:002   |8503202   |Zug           |10     |EC       |0.8202508288487688|0.014650408388357685 |11      |
| 15:14             |85:11:10016:007|8503000   |Zug           |10016  |IC       |0.8229764472710196|0.014770271373540382 |11      |
| 18:47             |85:11:10020:007|8503000   |Zug           |10020  |EC       |0.8842176300333565|0.009397486165561733 |11      |
| 21:17             |85:11:10022:015|8503000   |Zug           |10022  |IC       |0.8202508288487688|0.014650408388357685 |11      |
| 06:23             |85:11:1007:001 |8503000   |Zug           |1007   |IC4  

In [ ]:
probability.orderBy([''])

In [62]:
routes.dropDuplicates(['route_type']).show(100, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+---------+----------------+---------------+-------------+----------+
|route_id      |agency_id|route_short_name|route_long_name|route_desc   |route_type|
+--------------+---------+----------------+---------------+-------------+----------+
|20-20-A-j19-1 |11       |20              |               |Auoreisezug  |1700      |
|59-100-j19-1  |11       |100             |               |InterRegio   |103       |
|61-104-Y-j19-1|278      |104-Y           |               |Luftseilbahn |1300      |
|61-100-Y-j19-1|134      |100-Y           |               |Standseilbahn|1400      |
|5-1-j19-1     |823      |1               |               |Tram         |900       |
|12-1-B-j19-1  |11       |1               |               |Extrazug     |100       |
|90-100-Y-j19-1|181      |100-Y           |               |Schiff       |1000      |
|1-1-B-j19-1   |11       |1               |               |S-Bahn       |400       |
|1-1-j19-1     |11       |1               |               |Interc

In [33]:
s = (
    schedule
    .join(monday_trips, 'trip_id')
    .persist()
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
@f.udf
def operator(id_):
    parts = id_.split(':')
    if len(parts) > 1:
        return parts[1]
    else:
        return None

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [78]:
probability.withColumn('operator', operator(probability.trip_id)).orderBy(['operator', 'line_id']).where(f.col('operator') != "11").dropDuplicates(['transport_type']).show(100, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----------------------+----------+--------------+-----------+---------+------------------+--------------------+--------+
|arrival_time_minute|trip_id               |station_id|transport_type|line_id    |line_text|delay_probability |delay_parameter     |operator|
+-------------------+----------------------+----------+--------------+-----------+---------+------------------+--------------------+--------+
| 10:27             |85:3849:106781-14003-1|8591233   |Tram          |85:3849:003|3        |0.8585163819244139|0.016802126392468718|3849    |
| 01:23             |85:78:12400:001       |8503091   |Zug           |12400      |SN4      |0.8338964724962833|0.009546535686752945|78      |
| 04:55             |85:773:9238-01353-1   |8576239   |Bus           |85:773:353 |N45      |0.8797847964776709|0.015267638313938462|773     |
| 10:06             |85:3849:87595-02008-1 |8594239   |              |113852833  |T8       |0.8421052631578947|0.010342598577892695|3849    |
+-----

In [79]:
@f.udf
def train_identifier(trip_id):
    return ':'.join(trip_id.split(':')[:3])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [80]:
ident = (
    probability
    .withColumn('identifier', f.when(f.col('transport_type') == 'Zug', train_identifier(f.col('trip_id'))).otherwise(f.col('line_id')))
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [83]:
ident.groupBy(['identifier']).agg(f.countDistinct('transport_type').alias('count')).show(20)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-----+
| identifier|count|
+-----------+-----+
| 85:11:4937|    1|
|85:11:18383|    1|
|85:11:13775|    1|
|85:78:12910|    1|
|85:78:24796|    1|
|85:78:24447|    1|
|85:11:18862|    1|
|85:78:24630|    1|
|  85:31:626|    1|
|  85:11:816|    1|
|85:78:23514|    1|
|85:78:12448|    1|
| 85:11:2171|    1|
|85:78:24581|    1|
|       1512|    1|
|85:11:18965|    1|
|85:78:24935|    1|
| 85:11:2066|    1|
|85:11:18924|    1|
|  85:11:579|    1|
+-----------+-----+
only showing top 20 rows

In [54]:
probability.orderBy('trip_id', ascending=False).show(100, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------------+----------+--------------+--------+---------+------------------+--------------------+
|arrival_time_minute|trip_id       |station_id|transport_type|line_id |line_text|delay_probability |delay_parameter     |
+-------------------+--------------+----------+--------------+--------+---------+------------------+--------------------+
| 09:56             |85:886:99998-0|8590277   |Bus           |85:886:2|2        |0.9852139547560643|0.01685919268498297 |
| 09:59             |85:886:99998-0|8590279   |Bus           |85:886:2|2        |0.9852139547560643|0.01685919268498297 |
| 09:55             |85:886:99998-0|8590278   |Bus           |85:886:2|2        |0.9852139547560643|0.01685919268498297 |
| 09:57             |85:886:99998-0|8590271   |Bus           |85:886:2|2        |0.9852139547560643|0.01685919268498297 |
| 09:54             |85:886:99998-0|8590396   |Bus           |85:886:2|2        |0.9852139547560643|0.01685919268498297 |
| 08:15             |85:

In [52]:
probability.where(probability.trip_id.contains('10096')).show(100, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----------------------+----------+--------------+-----------+---------+------------------+--------------------+
|arrival_time_minute|trip_id               |station_id|transport_type|line_id    |line_text|delay_probability |delay_parameter     |
+-------------------+----------------------+----------+--------------+-----------+---------+------------------+--------------------+
| 18:17             |85:886:100969-0       |8590278   |Bus           |85:886:4   |4        |0.9882898806893504|0.007830411687439422|
| 07:00             |85:3849:210096-11014-1|8591276   |Tram          |85:3849:014|14       |0.8253783422406212|0.015651360199123544|
| 00:06             |85:3849:210096-22010-1|8590561   |Tram          |85:3849:012|12       |0.7791568296795953|0.02033122059488754 |
| 12:13             |85:773:100965-11733-1 |8580449   |Bus           |85:773:781 |781      |0.950653101582015 |0.010897492492510406|
| 13:39             |85:838:510096-20850-1 |8576097   |Bus           

In [37]:
s.orderBy(['trip_short_name', 'Stop_Station']).dropDuplicates(['trip_short_name']).show(1000, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------------+-------------+----------+-------------+---------+------------+----------+--------------+------------------------------+---------------+------------+
|Trip_ID                   |Stop_sequence|Start_Time|Start_Station|Stop_Time|Stop_Station|service_id|route_id      |trip_headsign                 |trip_short_name|direction_id|
+--------------------------+-------------+----------+-------------+---------+------------+----------+--------------+------------------------------+---------------+------------+
|529.TA.79-229-j19-1.1.R   |4            |18:12:00  |8579994      |18:13:00 |8573713     |TA+b0nfg  |79-229-j19-1  |Bonstetten-Wettswil, Bahnhof  |10096          |1           |
|437.TA.26-530-j19-1.1.R   |16           |16:15:00  |8582739      |16:16:00 |8503772     |TA+b0a0e  |26-530-j19-1  |Bülach, Bahnhof               |10351          |1           |
|943.TA.26-151-j19-1.3.R   |6            |19:09:00  |8590478      |19:11:00 |8590464     |TA+b002h  |26-151-j19-1  

In [28]:
s.show(20, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------+-------------+----------+-------------+---------+------------+----------+-----------+------------------------+---------------+------------+
|Trip_ID              |Stop_sequence|Start_Time|Start_Station|Stop_Time|Stop_Station|service_id|route_id   |trip_headsign           |trip_short_name|direction_id|
+---------------------+-------------+----------+-------------+---------+------------+----------+-----------+------------------------+---------------+------------+
|3.TA.1-217-j19-1.1.H |2            |06:45:00  |8583890      |06:48:00 |8573178     |TA+b0001  |1-217-j19-1|Affoltern a. A., Bahnhof|21707          |0           |
|3.TA.1-217-j19-1.1.H |1            |06:43:00  |8573195      |06:45:00 |8583890     |TA+b0001  |1-217-j19-1|Affoltern a. A., Bahnhof|21707          |0           |
|7.TA.1-217-j19-1.1.H |2            |08:45:00  |8583890      |08:48:00 |8573178     |TA+b0001  |1-217-j19-1|Affoltern a. A., Bahnhof|21715          |0           |
|7.TA.1-217-j19-1.1.H 

In [12]:
probability.sample(0.001).show(30, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----------------------+----------+--------------+-----------+---------+------------------+--------------------+
|arrival_time_minute|trip_id               |station_id|transport_type|line_id    |line_text|delay_probability |delay_parameter     |
+-------------------+----------------------+----------+--------------+-----------+---------+------------------+--------------------+
| 23:49             |85:3849:109091-03010-1|8591382   |Tram          |85:3849:010|10       |0.797695300585474 |0.015237956376985337|
| 12:38             |85:3849:141563-04007-1|8591106   |Tram          |85:3849:007|7        |0.8485443366175273|0.015191044341473024|
| 08:24             |85:3849:142888-04011-1|8591180   |Tram          |85:3849:011|11       |0.8421321645767593|0.01299529296167841 |
| 19:36             |85:3849:142915-04011-1|8530812   |Tram          |85:3849:011|11       |0.8228581490114534|0.01361149920342191 |
| 13:57             |85:3849:161437-14005-1|8591058   |Tram          

In [27]:
probability.where(probability.arrival_time_minute == ' 18:12').where(probability.station_id == '8587349').where(probability.line_text == '17').show(30, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----------------------+----------+--------------+-----------+---------+-----------------+-------------------+
|arrival_time_minute|trip_id               |station_id|transport_type|line_id    |line_text|delay_probability|delay_parameter    |
+-------------------+----------------------+----------+--------------+-----------+---------+-----------------+-------------------+
| 18:12             |85:3849:117317-04017-1|8587349   |Tram          |85:3849:017|17       |0.850921093628291|0.01045037426457373|
| 18:12             |85:3849:429826-33017-1|8587349   |Tram          |85:3849:017|17       |0.850921093628291|0.01045037426457373|
| 18:12             |85:3849:488537-14017-1|8587349   |Tram          |85:3849:017|17       |0.850921093628291|0.01045037426457373|
| 18:12             |85:3849:486620-13017-1|8587349   |Tram          |85:3849:017|17       |0.850921093628291|0.01045037426457373|
| 18:12             |85:3849:469751-14017-1|8587349   |Tram          |85:3849:017|1

In [37]:
schedule.where(schedule.Trip_ID.substr(0, 3) == '140').where(schedule.Stop_Station == '8587349').show(30, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+--------------------------+----------+-------------+---------+------------+
|Stop_sequence|Trip_ID                   |Start_Time|Start_Station|Stop_Time|Stop_Station|
+-------------+--------------------------+----------+-------------+---------+------------+
|15           |140.TA.26-17-j19-1.1.H    |16:26:00  |8591368      |16:28:00 |8587349     |
|15           |140.TA.26-7-B-j19-1.1.H   |13:25:00  |8591379      |13:27:00 |8587349     |
|13           |1400.TA.26-13-j19-1.20.R  |17:20:00  |8591067      |17:22:00 |8587349     |
|12           |1400.TA.26-14-A-j19-1.16.R|09:12:00  |8587348      |09:14:00 |8587349     |
|13           |1401.TA.26-13-j19-1.20.R  |17:13:00  |8591067      |17:15:00 |8587349     |
|12           |1401.TA.26-14-A-j19-1.16.R|08:57:00  |8587348      |08:59:00 |8587349     |
|13           |1402.TA.26-13-j19-1.20.R  |17:05:00  |8591067      |17:07:00 |8587349     |
|12           |1402.TA.26-14-A-j19-1.16.R|20:16:00  |8587348      |20:17:00 |8587349     |

In [34]:
schedule.where(schedule.Stop_Time == '18:12:00').where(schedule.Stop_Station == '8587349').show(30, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+------------------------+----------+-------------+---------+------------+
|Stop_sequence|Trip_ID                 |Start_Time|Start_Station|Stop_Time|Stop_Station|
+-------------+------------------------+----------+-------------+---------+------------+
|16           |1688.TA.26-13-j19-1.24.H|18:10:00  |8591368      |18:12:00 |8587349     |
|16           |1689.TA.26-13-j19-1.24.H|18:10:00  |8591368      |18:12:00 |8587349     |
|13           |776.TA.26-17-j19-1.19.R |18:09:00  |8591067      |18:12:00 |8587349     |
|13           |782.TA.26-17-j19-1.19.R |18:09:00  |8591067      |18:12:00 |8587349     |
|16           |79.TA.26-13-j19-1.1.H   |18:10:00  |8591368      |18:12:00 |8587349     |
|8            |1277.TA.26-17-j19-1.27.R|18:10:00  |8588078      |18:12:00 |8587349     |
|15           |138.TA.26-7-B-j19-1.1.H |18:10:00  |8591379      |18:12:00 |8587349     |
+-------------+------------------------+----------+-------------+---------+------------+

In [13]:
schedule.sample(0.001).show(30, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+-------------------------+----------+-------------+---------+------------+
|Stop_sequence|Trip_ID                  |Start_Time|Start_Station|Stop_Time|Stop_Station|
+-------------+-------------------------+----------+-------------+---------+------------+
|8            |299.TA.26-530-j19-1.2.R  |12:35:00  |8503857      |12:37:00 |8573209     |
|3            |299.TA.26-69-j19-1.3.R   |20:40:00  |8591425      |20:41:00 |8591419     |
|3            |2999.TA.26-8-C-j19-1.25.H|20:36:00  |8591131      |20:37:00 |8591135     |
|4            |3.TA.26-742-j19-1.1.H    |07:43:00  |8594236      |07:45:00 |8590765     |
|3            |3.TA.79-485-j19-1.1.R    |18:07:00  |8576249      |18:08:00 |8576250     |
|10           |30.TA.26-17-j19-1.1.H    |18:43:00  |8580522      |18:44:00 |8591110     |
|1            |30.TA.26-220-j19-1.1.H   |18:01:00  |8573710      |18:03:00 |8591341     |
|4            |30.TA.26-813-j19-1.2.R   |19:39:00  |8576788      |19:40:00 |8581539     |
|7        

In [15]:
stations = spark.read.csv("../data/zurich_stations_ids.csv")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
stations = (
    stations
    .select(stations._c0.alias('station_id'))
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
timetable = spark.read.csv("/user/gflueck/timetable.csv")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
u'Unable to infer schema for ORC. It must be specified manually.;'
Traceback (most recent call last):
  File "/hdata/sde/hadoop/yarn/local/usercache/ebouille/appcache/application_1589299642358_2070/container_e06_1589299642358_2070_01_000001/pyspark.zip/pyspark/sql/readwriter.py", line 476, in orc
    return self._df(self._jreader.orc(_to_seq(self._spark._sc, path)))
  File "/hdata/sde/hadoop/yarn/local/usercache/ebouille/appcache/application_1589299642358_2070/container_e06_1589299642358_2070_01_000001/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/hdata/sde/hadoop/yarn/local/usercache/ebouille/appcache/application_1589299642358_2070/container_e06_1589299642358_2070_01_000001/pyspark.zip/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
AnalysisException: u'Unable to infer schema for ORC. It must be specified manually.;'



In [ ]:
timetable = (
    timetable
    .join(f.broadcast(stations), timetable. how='right')
)

In [28]:
trips = spark.read.orc('/data/sbb/timetables/orc/trips')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
trips.where(trips.trip_id == '1688.TA.26-13-j19-1.24.H').show(20, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----------+------------------------+------------------+---------------+------------+
|route_id   |service_id|trip_id                 |trip_headsign     |trip_short_name|direction_id|
+-----------+----------+------------------------+------------------+---------------+------------+
|26-13-j19-1|TA+b0j2l  |1688.TA.26-13-j19-1.24.H|Zürich, Albisgütli|29954          |0           |
+-----------+----------+------------------------+------------------+---------------+------------+

In [43]:
routes = spark.read.orc('/data/sbb/timetables/orc/routes')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [113]:
routes.orderBy(['route_id']).show(20, False)

An error was encountered:
Invalid status code '404' from http://iccluster044.iccluster.epfl.ch:8998/sessions/7735 with error payload: "Session '7735' not found."


In [38]:
stop_times = spark.read.orc('/data/sbb/timetables/orc/stop_times')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
stop_times.orderBy(['trip_id', 'stop_sequence']).show(1000, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------+------------+--------------+---------------+-------------+-----------+-------------+
|trip_id               |arrival_time|departure_time|stop_id        |stop_sequence|pickup_type|drop_off_type|
+----------------------+------------+--------------+---------------+-------------+-----------+-------------+
|1.TA.1-1-A-j19-1.1.R  |01:07:00    |01:07:00      |8502113:0:4    |1            |0          |0            |
|1.TA.1-1-A-j19-1.1.R  |01:11:00    |01:11:00      |8502114:0:4    |2            |0          |0            |
|1.TA.1-1-A-j19-1.1.R  |01:15:00    |01:16:00      |8502119:0:2    |3            |0          |0            |
|1.TA.1-1-A-j19-1.1.R  |01:19:00    |01:19:00      |8502105:0:5    |4            |0          |0            |
|1.TA.1-1-A-j19-1.1.R  |01:22:00    |01:22:00      |8502247:0:5    |5            |0          |0            |
|1.TA.1-1-A-j19-1.1.R  |01:24:00    |01:24:00      |8502237:0:5    |6            |0          |0            |
|1.TA.1-1-A-j19-1.1

In [49]:
probability.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+---------------+----------+--------------+-------+---------+------------------+--------------------+
|arrival_time_minute|        trip_id|station_id|transport_type|line_id|line_text| delay_probability|     delay_parameter|
+-------------------+---------------+----------+--------------+-------+---------+------------------+--------------------+
|              21:42|   85:11:10:002|   8503202|           Zug|     10|       EC|0.8202508288487688|0.014650408388357685|
|              02:58|85:11:13714:001|   8503000|           Zug|  13714|      SN1|0.6519774011299435|0.006361772034664491|
|              02:12|85:11:13783:001|   8503000|           Zug|  13783|      SN8|0.8833798882681564|0.007784998646087192|
|              10:08|85:11:14037:001|   8503006|           Zug|  14037|      S24|0.8132095748704679|0.015289947865916156|
|              13:56|85:11:14051:002|   8503010|           Zug|  14051|      S24|  0.81328451151898|0.015783488520259153|
|              15:14|85:

In [59]:
(
    probability
    .where(probability.trip_id.substr(0, 14) == '85:3849:140841')
    .where(probability.arrival_time_minute == ' 11:55')
).show(20, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----------------------+----------+--------------+-----------+---------+------------------+--------------------+
|arrival_time_minute|trip_id               |station_id|transport_type|line_id    |line_text|delay_probability |delay_parameter     |
+-------------------+----------------------+----------+--------------+-----------+---------+------------------+--------------------+
| 11:55             |85:3849:140841-04004-1|8576198   |Tram          |85:3849:004|4        |0.8574072972731869|0.014359789452007226|
| 11:55             |85:3849:140841-04004-1|8576199   |Tram          |85:3849:004|4        |0.8574072972731869|0.014359789452007226|
| 11:55             |85:3849:140841-04004-1|8576193   |Tram          |85:3849:004|4        |0.8574072972731869|0.014359789452007226|
| 11:55             |85:3849:140841-04004-1|8576195   |Tram          |85:3849:004|4        |0.8574072972731869|0.014359789452007226|
| 11:55             |85:3849:140841-04004-1|8576200   |Tram          

In [67]:
stop_times.show(20, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------+------------+--------------+-----------+-------------+-----------+-------------+
|trip_id              |arrival_time|departure_time|stop_id    |stop_sequence|pickup_type|drop_off_type|
+---------------------+------------+--------------+-----------+-------------+-----------+-------------+
|1.TA.1-1-B-j19-1.1.R |04:20:00    |04:20:00      |8500010:0:3|1            |0          |0            |
|1.TA.1-1-B-j19-1.1.R |04:24:00    |04:24:00      |8500020:0:3|2            |0          |0            |
|1.TA.1-1-B-j19-1.1.R |04:28:00    |04:28:00      |8500021:0:5|3            |0          |0            |
|1.TA.1-1-B-j19-1.1.R |04:30:00    |04:30:00      |8517131:0:2|4            |0          |0            |
|1.TA.1-1-B-j19-1.1.R |04:32:00    |04:32:00      |8500300:0:5|5            |0          |0            |
|1.TA.1-1-B-j19-1.1.R |04:35:00    |04:35:00      |8500313:0:2|6            |0          |0            |
|1.TA.1-1-B-j19-1.1.R |04:37:00    |04:38:00      |8500301:0:3|7

In [60]:
from pyspark.sql import Window

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [89]:
order = Window().partitionBy('trip_id').orderBy('arrival_time')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [96]:
@f.udf
def station_hash(stations):
    return hash(tuple(stations))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [99]:
start_stop = (
    stop_times
    .withColumn('hash', station_hash(f.collect_list(stop_times.stop_id).over(order)))
    .withColumn('stop_id', f.last(stop_times.stop_id).over(order))
    # .withColumn('start_id_2', f.first(stop_times.stop_id).over(order))
    .persist()
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [100]:
start_stop.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------+--------------+------------+-------------+-----------+-------------+--------------------+
|             trip_id|arrival_time|departure_time|     stop_id|stop_sequence|pickup_type|drop_off_type|                hash|
+--------------------+------------+--------------+------------+-------------+-----------+-------------+--------------------+
|1.TA.11-12-B-j19-...|    08:30:00|      08:30:00|     8582915|            1|          0|            0|-4647608183507534734|
|1.TA.11-12-B-j19-...|    08:31:00|      08:31:00|     8582916|            2|          0|            0|-2344871266416595320|
|1.TA.11-12-B-j19-...|    08:33:00|      08:33:00|     8582917|            3|          0|            0| 8063082993366071779|
|1.TA.11-12-B-j19-...|    08:35:00|      08:35:00|     8574680|            4|          0|            0|   20657248049256456|
|1.TA.12-1-A-j19-1...|    01:30:00|      01:30:00|8505000:0:11|            1|          0|            0| 4230726535584772973|


In [ ]:
probability.groupBy(['trip_id', ''])

In [78]:
stop_times.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

11128930

In [79]:
start_stop.where(start_stop.start_id == start_stop.stop_id).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1033749

In [66]:
probability.groupBy(['arrival_time_minute', 'trip_id']).agg(f.collect_list(probability.station_id).alias('stations'), f.countDistinct(probability.station_id).alias('count')).where('count  > 5').show(20, False)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|arrival_time_minute|trip_id               |stations                                                                                                                                                                                     |count|
+-------------------+----------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
| 07:59             |85:3849:84551-02017-1 |[8591384, 8591329, 8591415, 8591059, 8591299, 8591404, 8591366]                                                                                                                              |7    |
| 11:55             |85:3849:140841-

In [101]:
sbb = spark.read.orc('/data/sbb/orc/istdaten')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [111]:
sbb.where(sbb.fahrt_bezeichner == '85:3849:140841-04004-1').where(sbb.ankunftszeit.substr(12, 5) == '11:55').dropDuplicates(['bpuic']).show(20, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----------------------+------------+-------------+-----------------------------+----------+-----------+-----------+---------+-------------------+--------------+-------------+-------+-----------------------+----------------+-------------------+------------------+----------------+-------------------+------------------+-------------+
|betriebstag|fahrt_bezeichner      |betreiber_id|betreiber_abk|betreiber_name               |produkt_id|linien_id  |linien_text|umlauf_id|verkehrsmittel_text|zusatzfahrt_tf|faellt_aus_tf|bpuic  |haltestellen_name      |ankunftszeit    |an_prognose        |an_prognose_status|abfahrtszeit    |ab_prognose        |ab_prognose_status|durchfahrt_tf|
+-----------+----------------------+------------+-------------+-----------------------------+----------+-----------+-----------+---------+-------------------+--------------+-------------+-------+-----------------------+----------------+-------------------+------------------+----------------+----------------